<a href="https://colab.research.google.com/github/dooby99/DogMedical/blob/main/U_NET_Modeling_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8_4_10%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Univ4-1

/content/drive/MyDrive/Univ4-1


In [ ]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms,datasets

dir_data = os.getcwd()
dir_data

'/content'

In [ ]:
### --- 테스트용 코드
_data_dir = os.path.join(dir_data, 'data')
_lst_data = os.listdir(_data_dir)

_lst_label = [f for f in _lst_data if f.startswith('label')]  # 문자열 검사해서 'label'이 있으면 True
_lst_input = [f for f in _lst_data if f.startswith('input')]  # 문자열 검사해서 'input'이 있으면 True

_lst_label

FileNotFoundError: ignored

In [ ]:
## 네트워크 구축
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=1, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)

        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_data = os.listdir(self.data_dir)

        lst_label = [f for f in lst_data if f.startswith('label')]  # 문자열 검사해서 'label'이 있으면 True
        lst_input = [f for f in lst_data if f.startswith('input')]  # 문자열 검사해서 'input'이 있으면 True

        lst_label.sort()
        lst_input.sort()

        self.lst_label = lst_label
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        inputs = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        # normalize
        label = label / 255.0
        inputs = inputs / 255.0
        label = label.astype(np.float32)
        inputs = inputs.astype(np.float32)

        # 인풋 데이터 차원이 2이면, 채널 축을 추가해줘야한다. 파이토치 인풋은 (batch, 채널, 행, 열)
        if label.ndim == 2:
            label = label[:, :, np.newaxis]
        if inputs.ndim == 2:
            inputs = inputs[:, :, np.newaxis]

        data = {'input': inputs, 'label': label}

        if self.transform:
            data = self.transform(data)  # transform에 할당된 class 들이 호출되면서 __call__ 함수 실행

        return data

In [ ]:
class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        label = label.transpose((0, 1, 2)).astype(np.float32)
        input = input.transpose((0, 1, 2)).astype(np.float32)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data

In [ ]:
class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        label, input = data['label'], data['input']

        input = (input - self.mean) / self.std

        data = {'label': label, 'input': input}

        return data

In [ ]:
class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            label = np.fliplr(label)
            input = np.fliplr(input)

        if np.random.rand() > 0.5:
            label = np.flipud(label)
            input = np.flipud(input)

        data = {'label': label, 'input': input}

        return data

In [ ]:
## 하이퍼 파라미터 설정

lr = 1e-3
batch_size = 10
num_epoch = 10

data_dir = os.path.join(dir_data, 'data')
ckpt_dir = os.path.join(dir_data, 'checkpoint')
log_dir = os.path.join(dir_data, 'log')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# transform 적용해서 데이터 셋 불러오기
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])
dataset_train = Dataset(data_dir=os.path.join(data_dir,'train'),transform=transform)

# 불러온 데이터셋, 배치 size줘서 DataLoader 해주기
loader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle=True)

# val set도 동일하게 진행
dataset_val = Dataset(data_dir=os.path.join(data_dir,'val'),transform = transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size , shuffle=True)

# 네트워크 불러오기
net = UNet().to(device) # device : cpu or gpu

# loss 정의
fn_loss = nn.BCEWithLogitsLoss().to(device)

# Optimizer 정의
optim = torch.optim.Adam(net.parameters(), lr = lr )

# 기타 variables 설정
num_train = len(dataset_train)
num_val = len(dataset_val)

num_train_for_epoch = np.ceil(num_train/batch_size) # np.ceil : 소수점 반올림
num_val_for_epoch = np.ceil(num_val/batch_size)

# 기타 function 설정
fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,1) # device 위에 올라간 텐서를 detach 한 뒤 numpy로 변환
fn_denorm = lambda x, mean, std : (x * std) + mean
fn_classifier = lambda x :  1.0 * (x > 0.5)  # threshold 0.5 기준으로 indicator function으로 classifier 구현

# Tensorbord
writer_train = SummaryWriter(log_dir=os.path.join(log_dir,'train'))
writer_val = SummaryWriter(log_dir = os.path.join(log_dir,'val'))

In [ ]:
# 네트워크 저장하기
# train을 마친 네트워크 저장
# net : 네트워크 파라미터, optim  두개를 dict 형태로 저장
def save(ckpt_dir,net,optim,epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(),'optim':optim.state_dict()},'%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# 네트워크 불러오기
def load(ckpt_dir,net,optim):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0
        return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str,isdigit,f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir,ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net,optim,epoch


# 네트워크 학습시키기
start_epoch = 0
net, optim, start_epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim) # 저장된 네트워크 불러오기

for epoch in range(start_epoch+1,num_epoch +1):
    net.train()
    loss_arr = []

    for batch, data in enumerate(loader_train, 1):
        # forward
        label = data['label'].to(device)   # 데이터 device로 올리기
        inputs = data['input'].to(device)
        output = net(inputs)

        # backward
        optim.zero_grad()  # gradient 초기화
        loss = fn_loss(output, label)  # output과 label 사이의 loss 계산
        loss.backward() # gradient backpropagation
        optim.step() # backpropa 된 gradient를 이용해서 각 layer의 parameters update

        # save loss
        loss_arr += [loss.item()]

        # tensorbord에 결과값들 저정하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs,0.5,0.5))
        output = fn_tonumpy(fn_classifier(output))

        writer_train.add_image('label', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)


    # validation
    with torch.no_grad(): # validation 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
        net.eval() # 네트워크를 evaluation 용으로 선언
        loss_arr = []

        for batch, data in enumerate(loader_val,1):
            # forward
            label = data['label'].to(device)
            inputs = data['input'].to(device)
            output = net(inputs)

            # loss
            loss = fn_loss(output,label)
            loss_arr += [loss.item()]
            print('valid : epoch %04d / %04d | Batch %04d \ %04d | Loss %04d'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_classifier(output))

            writer_val.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

        # epoch이 끝날때 마다 네트워크 저장
        save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)

writer_train.close()
writer_val.close()

valid : epoch 0001 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0001 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0002 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0002 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0003 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0003 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0004 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0004 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0005 / 0010 | Batch 0001 \ 0002 | Loss 0003
valid : epoch 0005 / 0010 | Batch 0002 \ 0002 | Loss 0003
valid : epoch 0006 / 0010 | Batch 0001 \ 0002 | Loss 0007
valid : epoch 0006 / 0010 | Batch 0002 \ 0002 | Loss 0006
valid : epoch 0007 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0007 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0008 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 0008 / 0010 | Batch 0002 \ 0002 | Loss 0000
valid : epoch 0009 / 0010 | Batch 0001 \ 0002 | Loss 0000
valid : epoch 

In [ ]:
pre_test = cv2.imread(os.path.join(os.getcwd(), "A1_구진_플라크", "IMG_D_A1_009835.jpg"), cv2.IMREAD_GRAYSCALE)
pre_test = cv2.resize(pre_test, dsize=(480, 320))
pre_test = np.array(pre_test)
cv2_imshow(pre_test)
pre_test = np.expand_dims(pre_test, axis=0)

net.eval()
masks = net(pre_test)
cv2_imshow(masks)

error: ignored

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [ ]:
import json
import os
import cv2
import numpy as np
from PIL import Image

from google.colab.patches import cv2_imshow

In [ ]:
with open(os.getcwd() + '/temp_image/IMG_D_A1_000001.json', 'r', encoding='UTF8') as f:
    json_data = json.load(f)

json_data

FileNotFoundError: ignored

In [ ]:
dir_data = os.getcwd()

_data_dir = os.path.join(dir_data, 'A1_구진_플라크')
_lst_data = os.listdir(_data_dir)

_lst_label = [f for f in _lst_data if f.endswith('json')] # only json list
cnt = 0

FileNotFoundError: ignored

In [ ]:
cnt = 0
div_size = 4

for fileUrl in _lst_label:

    fileUrl = os.path.join(_data_dir, fileUrl)

    # 지워야함
    if cnt > 10:
        break

    with open(fileUrl, 'r', encoding='UTF8') as f:

        json_data = json.load(f)
        image_name = json_data['metaData']['Raw data ID']
        info = json_data['labelingInfo']
        color = int(info[0]['polygon']['color'][1:], 16)

        try:

            ## Input data
            origin_img = cv2.imread(os.path.join(_data_dir, image_name), cv2.IMREAD_GRAYSCALE)
            origin_img_size = np.array(origin_img).shape

            resized_img = cv2.resize(origin_img, dsize=(480, 320))
            cv2_imshow(resized_img)
            resized_img = np.array(resized_img)
            resized_img = np.expand_dims(resized_img, axis=0)

            print(resized_img.shape)
            print(image_name)

            np.save(os.path.join(dir_data, 'data', 'train', 'input' + str(cnt)), np.array(resized_img))
            np.save(os.path.join(dir_data, 'data', 'val', 'input' + str(cnt)), np.array(resized_img))

            ## Label data
            label_mask = np.zeros((3,)+(480, 320), dtype=np.float64)
            label_mask = np.swapaxes(label_mask, 0, 2)
            print(label_mask.shape) # (320, 480, 3)

            polygon_location_info = info[0]['polygon']['location'][0]

            points_cnt = int(len(polygon_location_info)/2)
            tmp_dots = np.empty((points_cnt, 2), int)

            for key, value in zip(polygon_location_info.keys(), polygon_location_info.values()):

                pos = int(key[1:])-1

                if key.startswith('x'):
                    tmp_dots[pos][0] = int(value/div_size)
                else:
                    tmp_dots[pos][1] = int(value/origin_img_size[0]*320)

            label_mask = cv2.fillPoly(label_mask.copy(), [tmp_dots], (100, 100, 100))
            cv2_imshow(label_mask)
            label_mask = np.array(label_mask).sum(axis=2)
            label_mask = np.expand_dims(label_mask, axis=0)
            print(label_mask.shape)

            np.save(os.path.join(dir_data, 'data', 'train', 'label' + str(cnt)), np.array(label_mask))
            np.save(os.path.join(dir_data, 'data', 'val', 'label' + str(cnt)), np.array(label_mask))

            cnt += 1
            print(cnt)

        except:
            continue;

        # np.save(os.path.join(os.getcwd(), 'data', 'train', 'input' + str(index)), image_np)
        # np.save(os.path.join(os.getcwd(), 'data', 'val', 'input' + str(index)), image_np)

NameError: ignored